In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [4]:
n_rows = 10000000

train = pd.read_csv(
    '../input/train.csv.zip', 
    index_col='key', 
    compression='infer', 
    nrows=n_rows)

valid = pd.read_csv(
    '../input/train.csv.zip', 
    index_col='key', 
    compression='infer', 
    nrows=n_rows/10, 
    skiprows=list(range(1,n_rows+1)))

infer = pd.read_csv('../input/test.csv', index_col='key')

In [5]:
def feature_engineering(df):
    df['pickup'] = pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S %Z')
    df['weekday'] = df['pickup'].dt.dayofweek
    df['month'] = df['pickup'].dt.month
    df['hour'] = df['pickup'].dt.hour
    df['year'] = df['pickup'].dt.year
        
    df['lat_diff'] = abs(df['pickup_latitude'] - df['dropoff_latitude'])
    df['lon_diff'] = abs(df['pickup_longitude'] - df['dropoff_longitude'])
    df['taxicab'] = df['lat_diff'] + df['lon_diff']
        
    df.drop(['pickup_datetime','pickup'], axis=1, inplace=True)
    
feature_engineering(train)
feature_engineering(valid)
feature_engineering(infer)

train.dropna(how='any', axis=0, inplace=True)
valid.dropna(how='any', axis=0, inplace=True)

In [6]:
y_train = train['fare_amount']
y_valid = valid['fare_amount']

X_train = train.drop('fare_amount', axis=1, inplace=True)
X_valid = valid.drop('fare_amount', axis=1, inplace=True)

In [7]:
# definition of the numeric columns
p_lat = tf.feature_column.numeric_column('pickup_latitude')
p_lon = tf.feature_column.numeric_column('pickup_longitude')
d_lat = tf.feature_column.numeric_column('dropoff_latitude')
d_lon = tf.feature_column.numeric_column('dropoff_longitude')
p_cnt = tf.feature_column.numeric_column('passenger_count')
lat_diff = tf.feature_column.numeric_column('lat_diff')
lon_diff = tf.feature_column.numeric_column('lon_diff')
taxicab = tf.feature_column.numeric_column('taxicab')

# definition of the categorical columns
weekday = tf.feature_column.categorical_column_with_identity('weekday', num_buckets = 7)
hour = tf.feature_column.categorical_column_with_identity('hour', num_buckets = 24)
month = tf.feature_column.categorical_column_with_identity('month', num_buckets = 13)
year = tf.feature_column.categorical_column_with_identity('year', num_buckets = 2016)

# definition of the bucketized columns
numbuckets = 16

latbuckets = np.linspace(38,42,numbuckets).tolist()
lonbuckets = np.linspace(-76,-72,numbuckets).tolist()

p_latB = tf.feature_column.bucketized_column(p_lat, latbuckets)
d_latB = tf.feature_column.bucketized_column(d_lat, latbuckets)
p_lonB = tf.feature_column.bucketized_column(p_lon, lonbuckets)
d_lonB = tf.feature_column.bucketized_column(d_lon, lonbuckets)

# definition of the feature crosses
ploc = tf.feature_column.crossed_column([p_latB, p_lonB], numbuckets * numbuckets)
dloc = tf.feature_column.crossed_column([d_latB, d_lonB], numbuckets * numbuckets)
pd_pair = tf.feature_column.crossed_column([ploc, dloc], numbuckets**4)
day_hr = tf.feature_column.crossed_column([weekday, hour], 7 * 24)
mth_yr = tf.feature_column.crossed_column([month, year], 12 * 10)

In [8]:
wide_columns = [
    # Feature crosses
    ploc, dloc, pd_pair, day_hr, mth_yr,

    # Sparse columns
    weekday, hour,

    # Anything with a linear relationship
    p_cnt 
]

In [9]:
deep_columns = [
    # Embedding_column to "group" together ...
    tf.feature_column.embedding_column(pd_pair, 10),
    tf.feature_column.embedding_column(day_hr, 10),
    tf.feature_column.embedding_column(mth_yr, 10),

    # Numeric columns
    p_lat, p_lon, d_lat, d_lon, lat_diff, lon_diff, taxicab
]

In [10]:
def train_input_fn(features, labels, batch_size):
    return tf.estimator.inputs.pandas_input_fn(
        x = features,
        y = labels,
        num_epochs = 1,
        shuffle = True,
        batch_size = batch_size)

def eval_input_fn(features, labels):
    return tf.estimator.inputs.pandas_input_fn(
        x = features,
        y = labels,
        num_epochs = 1,
        shuffle = True)

def pred_input_fn(features):
    return tf.estimator.inputs.pandas_input_fn(
        x = features,
        num_epochs = 1,
        shuffle = False)

In [11]:
model_dir = './DNN_Linear_Combined_Regressor'

file_writer = tf.summary.FileWriter(model_dir)

sess_config = tf.ConfigProto(log_device_placement=True)
run_config = tf.estimator.RunConfig(session_config = sess_config)

estimator = tf.estimator.DNNLinearCombinedRegressor(
    model_dir = model_dir,
    linear_feature_columns = wide_columns,
    dnn_feature_columns = deep_columns,
    dnn_optimizer = tf.train.AdamOptimizer(learning_rate=0.0001),
    dnn_hidden_units = [64,64,64,8], 
    batch_norm = True,
    dnn_dropout = 0.1,
    config=run_config
)

INFO:tensorflow:Using config: {'_model_dir': './DNN_Linear_Combined_Regressor', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': log_device_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000018A2E9FD9E8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [12]:
train_spec = tf.estimator.TrainSpec(input_fn = train_input_fn(train, y_train, batch_size=10000))
eval_spec = tf.estimator.EvalSpec(input_fn = eval_input_fn(valid, y_valid))

In [13]:
for i in range(100):
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./DNN_Linear_Combined_Regressor\model.ckpt.
INFO:tensorflow:loss = 2208531.2, step = 0
INFO:tensorflow:global_step/sec: 6.17785
INFO:tensorflow:loss = 1929313.5, step = 100 (16.189 sec)
INFO:tensorflow:global_step/sec: 6.64862
INFO:tensorflow:loss = 1987361.5, step = 200 (15.039 sec)
INFO:tensorflow:global_step/sec: 7.16766
INFO:tensorflow:loss = 1749178.5, step = 300 (13.952 sec)
INFO:tensorflow:global_step/sec: 7.1795

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3000: ./DNN_Linear_Combined_Regressor\model.ckpt-3000
INFO:tensorflow:Loss for final step: 593406.94.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_Linear_Combined_Regressor\model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 3000 into ./DNN_Linear_Combined_Regressor\model.ckpt.
INFO:tensorflow:loss = 596278.94, step = 3000
INFO:tensorflow:global_step/sec: 6.14399
INFO:tensorflow:loss = 492598.88

INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-08-14-02:10:12
INFO:tensorflow:Saving dict for global step 6000: average_loss = 69.94392, global_step = 6000, label/mean = 11.297571, loss = 8952.821, prediction/mean = 11.049648
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 6000: ./DNN_Linear_Combined_Regressor\model.ckpt-6000
INFO:tensorflow:Loss for final step: 399983.94.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_Linea

INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-08-14-02:19:09
INFO:tensorflow:Saving dict for global step 9000: average_loss = 58.402794, global_step = 9000, label/mean = 11.250606, loss = 7475.5576, prediction/mean = 10.662148
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 9000: ./DNN_Linear_Combined_Regressor\model.ckpt-9000
INFO:tensorflow:Loss for final step: 273355.56.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determin

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-14-02:27:56
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_Linear_Combined_Regressor\model.ckpt-12000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-08-14-02:28:09
INFO:tensorflow:Saving dict for global step 12000: average_loss = 57.388504, global_step = 12000, label/mean = 11.367099, loss = 7345.7285, prediction/mean = 10.317262
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1200

INFO:tensorflow:loss = 245838.9, step = 14700 (15.091 sec)
INFO:tensorflow:global_step/sec: 6.79467
INFO:tensorflow:loss = 390366.22, step = 14800 (14.717 sec)
INFO:tensorflow:global_step/sec: 6.90258
INFO:tensorflow:loss = 1873153.4, step = 14900 (14.487 sec)
INFO:tensorflow:Saving checkpoints for 15000 into ./DNN_Linear_Combined_Regressor\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-14-02:37:12
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_Linear_Combined_Regressor\model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflo

UnknownError: Failed to rename: ./DNN_Linear_Combined_Regressor\model.ckpt-17000_temp_5be0702f7bde468ab53e085698c3b92e/part-00000-of-00002.data-00000-of-00001 to: ./DNN_Linear_Combined_Regressor\model.ckpt-17000.data-00000-of-00002 : Access is denied.
; Input/output error
	 [[Node: save/MergeV2Checkpoints = MergeV2Checkpoints[delete_old_dirs=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](save/MergeV2Checkpoints/checkpoint_prefixes, _arg_save/Const_0_0)]]

Caused by op 'save/MergeV2Checkpoints', defined at:
  File "C:\Users\vasil\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\vasil\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\vasil\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\vasil\AppData\Local\Continuum\anaconda3\lib\site-pac

In [14]:
generator = estimator.predict(input_fn=pred_input_fn(infer))
predictions = [next(generator) for i in range(len(infer))]
values = [val['predictions'].tolist()[0] for val in predictions]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_Linear_Combined_Regressor\model.ckpt-17000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [16]:
ids = infer.index

submission = pd.DataFrame()
submission['fare_amount'] = values
submission['key'] = ids
submission.set_index('key', inplace=True)

submission.to_csv('../output/03.dnn_linear_combined_regressor_r2.csv')